In [ ]:
%matplotlib inline

##데이터 작업하기

파이토치는 DataLoader & Dataset으로 구성

Dataset - 샘플 & 정답(label)을 저장
Dataloader - dataset을 순회 가능한(데이터를 하나씩 처리 가능한) 객체로 감싼다.

In [ ]:
# pytorch의 도메인특화 데이터 셋 - 라이브러리 중 torchvision 사용

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

데이터를 불러오며 경로지정 (root) , 데이터 종류 결정 (train),
다운로드 여부 (download), 변형방법 지정 (transform) 진행

In [ ]:
# 학습데이터 받기

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# 테스트 데이터 받기

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

dataset을 dataloader의 인자로 전달

dataloader에서 자동화 배치, 샘플링, 섞기 등 multiprocess data loading 지원

batch size : 64 -> dataloader 객체의 각 요소는 64개의 배치 (feature -label)로 반환

In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader: # train 도 동일
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

## 모델 만들기

파이토치에서 신경망 모델은 nn.module 을 상속받는 class 생성해서 정의

init 함수에서는 신경망의 계층(layer)정의,
forward 함수에서는 신경망에 데이터를 어떻게 전달하지 지정

GPU 혹은 MPS 로 신경망 이동, 연산을 가속

In [ ]:
#학습에 사용되는 cpu나 gpu, mps 얻기

device = (
    'cuda'
    if torch.cuda.is_available() # 사용가능 여부 확인
    else 'mps'
    if torch.backends.mps.is_available() # 사용가능 여부 확인
    else 'cpu'
)
print(f"Using {device} device")

# 모델 정의

class NeuralNetwork(nn.Module):
    def __init__(self): # 초기화 메소드
        super().__init__() # 부모의 초기화 메소드 호출
        self.flatten = nn.Flatten() # Flatten 객체 생성
        self.linear_relu_stack = nn.Sequential( #시퀀셜 레이어 설정 (선형 레이어, 활성함수 X 3개로 구성)
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x): # 순전파 메소드 설정
        x = self.flatten(x) # 입력 데이터 x를 평탄화로 1d 벡터로 변환
        logits = self.linear_relu_stack(x) # 시퀀셜 레이어로 순전파
        return logits

model = NeuralNetwork().to(device)
print(model)

## 모델 매개변수 최적화

모델 학습을 위해 손실함수 & 옵티마이저 제작

In [ ]:
loss_fn = nn.CrossEntropyLoss() # 손실함수 정의
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # 옵티마이저 정의

학습함수 정의

각 학습단계 (training loop) 에서 예측 수행 및 예측 오류를 역전파해 매개변수 조정  

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # 전체 데이터셋 크기를 가져옴
    for batch, (X, y) in enumerate(dataloader): # dataloader에서 데이터 받아오기
        X, y = X.to(device), y.to(device) # cpu로 옮기기

        # 예측 오류 계산
        pred = model(X) # 예측값 계산
        loss = loss_fn(pred, y) # 차이 계산

        # 역전파
        optimizer.zero_grad() # 기울기 초기화
        loss.backward() # 역전파 수행
        optimizer.step() # 파라미터 업데이트 -> 역전파된 기울기로 최적화 알고리즘 수행

        if batch % 100 == 0: # 100번째 마다 진행상황출력
            loss, current = loss.item(), batch * len(X) #
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

테스트 함수 정의
모델의 성능을 본다.

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # 모델을 평가모드로 전환  드롭아웃 같은 기능 off
    test_loss, correct = 0, 0 # 손실값, 정확도 초기화
    with torch.no_grad(): # 기울기 계산 비활성화 (역전파 x)
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X) # 예측 수행
            test_loss += loss_fn(pred, y).item() # 테스트 손실값, 미니배치 손실 더하기 ??
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # 정확히 예측한 값 누적
    test_loss /= num_batches # 비율 계산
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

반복단계(epoch)을 거쳐 각 에폭마다 정확도- 손실 출력

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

##모델 저장하기
내부 상태 dictionary 를 직렬화


In [ ]:
torch .save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

##모델 불러오기

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval() # 평가모드
x, y = test_data[0][0], test_data[0][1] # 입력 데이터 & 해당 레이블 가져오기
with torch.no_grad(): # 역전파 off
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted : "{predicted}", Actual: "{actual}"')